In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,7,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,222,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,223,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,583,"CREATE TABLE accounts (\tid integer,\tname bpc..."


![ERD](PaP_ERD.png)

Task 1:

Provide the name of the sales_rep in each region with the largest amount of total_amt_usd sales.

In [6]:
pd.read_sql_query(sql='''
SELECT * FROM(SELECT r.name, s.name, SUM(total_amt_usd)
FROM orders o
JOIN accounts a
ON o.account_id = a.id
JOIN sales_reps s
ON a.sales_rep_id = s.id
JOIN region r
ON s.region_id = r.id
GROUP BY 1, 2
ORDER BY 1, 3 DESC)
GROUP BY 1;
''', con=conn)

,name,name:1,SUM(total_amt_usd)
0,Midwest,Charles Bidwell,675637.19
1,Northeast,Tia Amato,1010690.60
2,Southeast,Earlie Schleusner,1098137.72
3,West,Georgianna Chisholm,886244.12


In [7]:
pd.read_sql_query(sql='''
SELECT region_name, rep_name, MAX(total_amt) total_amt
     FROM(SELECT s.name rep_name, r.name region_name, SUM(o.total_amt_usd) total_amt
             FROM sales_reps s
             JOIN accounts a
             ON a.sales_rep_id = s.id
             JOIN orders o
             ON o.account_id = a.id
             JOIN region r
             ON r.id = s.region_id
             GROUP BY 1, 2) t1
     GROUP BY 1;
''', con=conn)

,region_name,rep_name,total_amt
0,Midwest,Charles Bidwell,675637.19
1,Northeast,Tia Amato,1010690.60
2,Southeast,Earlie Schleusner,1098137.72
3,West,Georgianna Chisholm,886244.12


In [8]:
# my horror solution, most universal and recommended
# t2 is region anf max value, t3 adds sales rep detail

pd.read_sql_query(sql='''
SELECT t3.* FROM
(SELECT region_name, MAX(total_amt) total_amt
FROM (SELECT s.name rep_name, r.name region_name, SUM(o.total_amt_usd) total_amt
     FROM sales_reps s
     JOIN accounts a
     ON a.sales_rep_id = s.id
     JOIN orders o
     ON o.account_id = a.id
     JOIN region r
     ON r.id = s.region_id
     GROUP BY 1, 2) t1
GROUP BY 1) t2
JOIN (SELECT s.name rep_name, r.name region_name, SUM(o.total_amt_usd) total_amt
     FROM sales_reps s
     JOIN accounts a
     ON a.sales_rep_id = s.id
     JOIN orders o
     ON o.account_id = a.id
     JOIN region r
     ON r.id = s.region_id
     GROUP BY 1, 2) t3
ON t2.total_amt = t3.total_amt
;
''', con=conn)

,rep_name,region_name,total_amt
0,Charles Bidwell,Midwest,675637.19
1,Tia Amato,Northeast,1010690.60
2,Earlie Schleusner,Southeast,1098137.72
3,Georgianna Chisholm,West,886244.12


In [9]:
# horror, but most universal and recommended

pd.read_sql_query(sql='''
SELECT t3.rep_name, t3.region_name, t3.total_amt
FROM(SELECT region_name, MAX(total_amt) total_amt
     FROM(SELECT s.name rep_name, r.name region_name, SUM(o.total_amt_usd) total_amt
             FROM sales_reps s
             JOIN accounts a
             ON a.sales_rep_id = s.id
             JOIN orders o
             ON o.account_id = a.id
             JOIN region r
             ON r.id = s.region_id
             GROUP BY 1, 2) t1
     GROUP BY 1) t2
JOIN (SELECT s.name rep_name, r.name region_name, SUM(o.total_amt_usd) total_amt
     FROM sales_reps s
     JOIN accounts a
     ON a.sales_rep_id = s.id
     JOIN orders o
     ON o.account_id = a.id
     JOIN region r
     ON r.id = s.region_id
     GROUP BY 1,2
     ORDER BY 3 DESC) t3
ON t3.region_name = t2.region_name AND t3.total_amt = t2.total_amt;
''', con=conn)

,rep_name,region_name,total_amt
0,Charles Bidwell,Midwest,675637.19
1,Tia Amato,Northeast,1010690.60
2,Earlie Schleusner,Southeast,1098137.72
3,Georgianna Chisholm,West,886244.12


Solution:

SELECT region_name, MAX(total_amt) total_amt
     FROM(SELECT s.name rep_name, r.name region_name, SUM(o.total_amt_usd) total_amt
             FROM sales_reps s
             JOIN accounts a
             ON a.sales_rep_id = s.id
             JOIN orders o
             ON o.account_id = a.id
             JOIN region r
             ON r.id = s.region_id
             GROUP BY 1, 2) t1
     GROUP BY 1;

Task 2:

For the region with the largest (sum) of sales total_amt_usd, how many total (count) orders were placed?

In [10]:
pd.read_sql_query(sql='''
SELECT r.name, SUM(total_amt_usd), COUNT (*)
FROM orders o
JOIN accounts a
ON o.account_id = a.id
JOIN sales_reps s
ON a.sales_rep_id = s.id
JOIN region r
ON s.region_id = r.id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1;
''', con=conn)

,name,SUM(total_amt_usd),COUNT (*)
0,Northeast,7744405.36,2357


In [11]:
pd.read_sql_query(sql='''
SELECT r.name, COUNT(o.total) total_orders
FROM sales_reps s
JOIN accounts a
ON a.sales_rep_id = s.id
JOIN orders o
ON o.account_id = a.id
JOIN region r
ON r.id = s.region_id
GROUP BY r.name
HAVING SUM(o.total_amt_usd) = (
      SELECT MAX(total_amt)
      FROM (SELECT r.name region_name, SUM(o.total_amt_usd) total_amt
              FROM sales_reps s
              JOIN accounts a
              ON a.sales_rep_id = s.id
              JOIN orders o
              ON o.account_id = a.id
              JOIN region r
              ON r.id = s.region_id
              GROUP BY r.name) sub);
''', con=conn)

,name,total_orders
0,Northeast,2357


Solution:

SELECT r.name, COUNT(o.total) total_orders
FROM sales_reps s
JOIN accounts a
ON a.sales_rep_id = s.id
JOIN orders o
ON o.account_id = a.id
JOIN region r
ON r.id = s.region_id
GROUP BY r.name
HAVING SUM(o.total_amt_usd) = (
      SELECT MAX(total_amt)
      FROM (SELECT r.name region_name, SUM(o.total_amt_usd) total_amt
              FROM sales_reps s
              JOIN accounts a
              ON a.sales_rep_id = s.id
              JOIN orders o
              ON o.account_id = a.id
              JOIN region r
              ON r.id = s.region_id
              GROUP BY r.name) sub);

Task 3:

How many accounts had more total purchases than the account name which has bought the most standard_qty paper throughout their lifetime as a customer?

In [12]:
pd.read_sql_query(sql='''
SELECT t1.*, t2.*
FROM
(SELECT a.name, SUM(standard_qty) standard, SUM(total) total
FROM orders o
JOIN accounts a
ON o.account_id = a.id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1) t1
JOIN (SELECT a.name, SUM(total) total
FROM orders o
JOIN accounts a
ON o.account_id = a.id
GROUP BY 1
ORDER BY 2 DESC) t2
ON t2.total > t1.total;
''', con=conn)

,name,standard,total,name,total
0,Core-Mark Holding,41617,44750,EOG Resources,56410
1,Core-Mark Holding,41617,44750,Mosaic,49246
2,Core-Mark Holding,41617,44750,IBM,47506


In [13]:
# get names
pd.read_sql_query(sql='''
SELECT a.name, SUM(o.total)
       FROM orders o
       JOIN accounts a
       ON a.id = o.account_id
       GROUP BY 1
       HAVING SUM(o.total) > (SELECT total 
                   FROM (SELECT a.name act_name, SUM(o.standard_qty) tot_std, SUM(o.total) total
                         FROM accounts a
                         JOIN orders o
                         ON o.account_id = a.id
                         GROUP BY 1
                         ORDER BY 2 DESC
                         LIMIT 1) inner_tab);
''', con=conn)

,name,SUM(o.total)
0,EOG Resources,56410
1,IBM,47506
2,Mosaic,49246


In [14]:
# get count
pd.read_sql_query(sql='''
SELECT COUNT(*)
FROM (SELECT a.name
       FROM orders o
       JOIN accounts a
       ON a.id = o.account_id
       GROUP BY 1
       HAVING SUM(o.total) > (SELECT total 
                   FROM (SELECT a.name act_name, SUM(o.standard_qty) tot_std, SUM(o.total) total
                         FROM accounts a
                         JOIN orders o
                         ON o.account_id = a.id
                         GROUP BY 1
                         ORDER BY 2 DESC
                         LIMIT 1) inner_tab)
             ) counter_tab;
''', con=conn)

,COUNT(*)
0,3


Solution:

SELECT COUNT(*)
FROM (SELECT a.name
       FROM orders o
       JOIN accounts a
       ON a.id = o.account_id
       GROUP BY 1
       HAVING SUM(o.total) > (SELECT total 
                   FROM (SELECT a.name act_name, SUM(o.standard_qty) tot_std, SUM(o.total) total
                         FROM accounts a
                         JOIN orders o
                         ON o.account_id = a.id
                         GROUP BY 1
                         ORDER BY 2 DESC
                         LIMIT 1) inner_tab)
             ) counter_tab;

Task 4:

For the customer that spent the most (in total over their lifetime as a customer) total_amt_usd, how many web_events did they have for each channel?

In [15]:
pd.read_sql_query(sql='''
SELECT *
FROM (SELECT a.name name, SUM(o.total)
FROM accounts a
JOIN orders o
ON o.account_id = a.id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1) inner_tab;
''', con=conn)

,name,SUM(o.total)
0,EOG Resources,56410


In [16]:
pd.read_sql_query(sql='''
SELECT a.name, w.channel, COUNT(*) 
FROM accounts a
JOIN web_events w
ON w.account_id = a.id
GROUP BY 1, 2
HAVING (a.name = 
(SELECT name
FROM (SELECT a.name name, SUM(o.total)
FROM accounts a
JOIN orders o
ON o.account_id = a.id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1) inner_tab))
ORDER BY 3 DESC;
''', con=conn)

,name,channel,COUNT(*)
0,EOG Resources,direct,44
1,EOG Resources,organic,13
2,EOG Resources,adwords,12
3,EOG Resources,facebook,11
4,EOG Resources,twitter,5
5,EOG Resources,banner,4


In [17]:
# nice solution!!!

pd.read_sql_query(sql='''
SELECT a.name, w.channel, COUNT(*)
FROM accounts a
JOIN web_events w
ON a.id = w.account_id AND a.id =  (SELECT id
                     FROM (SELECT a.id, a.name, SUM(o.total_amt_usd) tot_spent
                           FROM orders o
                           JOIN accounts a
                           ON a.id = o.account_id
                           GROUP BY a.id, a.name
                           ORDER BY 3 DESC
                           LIMIT 1) inner_table)
GROUP BY 1, 2
ORDER BY 3 DESC;
''', con=conn)

,name,channel,COUNT(*)
0,EOG Resources,direct,44
1,EOG Resources,organic,13
2,EOG Resources,adwords,12
3,EOG Resources,facebook,11
4,EOG Resources,twitter,5
5,EOG Resources,banner,4


Solution:

SELECT a.name, w.channel, COUNT(*)
FROM accounts a
JOIN web_events w
ON a.id = w.account_id AND a.id =  (SELECT id
                     FROM (SELECT a.id, a.name, SUM(o.total_amt_usd) tot_spent
                           FROM orders o
                           JOIN accounts a
                           ON a.id = o.account_id
                           GROUP BY a.id, a.name
                           ORDER BY 3 DESC
                           LIMIT 1) inner_table)
GROUP BY 1, 2
ORDER BY 3 DESC;

Task 5:

What is the lifetime average amount spent in terms of total_amt_usd for the top 10 total spending accounts?

In [21]:
pd.read_sql_query(sql='''
SELECT AVG(total_spent) average FROM
(SELECT a.id id, a.name name, SUM(o.total_amt_usd) total_spent
FROM accounts a
JOIN orders o
ON o.account_id = a.id
GROUP BY 1, 2
ORDER BY 3 DESC
LIMIT 10) temp;
''', con=conn)

,average
0,304846.969


Task 6:

What is the lifetime average amount spent in terms of total_amt_usd, including only the companies that spent more per order, on average, than the average of all orders.

In [31]:
pd.read_sql_query(sql='''
SELECT AVG(total_spent) average FROM
(SELECT a.id id, a.name name, SUM(o.total_amt_usd) total_spent, AVG(total_amt_usd) average
FROM accounts a
JOIN orders o
ON o.account_id = a.id
GROUP BY 1, 2
HAVING average > (SELECT AVG(total_amt_usd) average
FROM orders)
ORDER BY 3 DESC) temp;
''', con=conn)

,average
0,89733.773787


In [28]:
# part
pd.read_sql_query(sql='''
SELECT a.id id, a.name name, SUM(o.total_amt_usd) total_spent, AVG(total_amt_usd) average
FROM accounts a
JOIN orders o
ON o.account_id = a.id
GROUP BY 1, 2
HAVING average > (SELECT AVG(total_amt_usd) average
FROM orders)
ORDER BY 3 DESC;
''', con=conn)

,id,name,total_spent,average
0,4211,EOG Resources,382873.30,6175.375806
1,4151,Mosaic,345618.59,5236.645303
2,1301,IBM,326819.48,5446.991333
3,1871,General Dynamics,300694.79,4555.981667
4,4111,Republic Services,293861.14,5877.222800
...,...,...,...,...
164,3841,W.W. Grainger,8384.61,4192.305000
165,1861,Publix Super Markets,8364.40,4182.200000
166,2321,Danaher,8133.47,4066.735000
167,1031,Berkshire Hathaway,7474.32,7474.320000


In [27]:
# part
pd.read_sql_query(sql='''
SELECT AVG(total_amt_usd) average
FROM orders;
''', con=conn)

,average
0,3348.019651


In [29]:
pd.read_sql_query(sql='''
SELECT AVG(avg_amt)
FROM (SELECT o.account_id, AVG(o.total_amt_usd) avg_amt
    FROM orders o
    GROUP BY 1
    HAVING AVG(o.total_amt_usd) > (SELECT AVG(o.total_amt_usd) avg_all
                                   FROM orders o)) temp_table;
                                   ''', con=conn)

,AVG(avg_amt)
0,4721.139744


Solution:

SELECT AVG(avg_amt)
FROM (SELECT o.account_id, AVG(o.total_amt_usd) avg_amt
    FROM orders o
    GROUP BY 1
    HAVING AVG(o.total_amt_usd) > (SELECT AVG(o.total_amt_usd) avg_all
                                   FROM orders o)) temp_table;